# Création automatique des tables de dimensions

In [1]:
from scripts.config import *
import glob
import pandas as pd

In [2]:
print(DATA_ORIGINAL_PATIENT)

path_to_data = DATA_ORIGINAL_PATIENT

files = glob.glob('{}*.xlsx'.format(path_to_data))

/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/patient-donor/


In [13]:
file = pd.ExcelFile(files[0])
sheets = file.sheet_names
output = {}
for sheet in sheets:
    if sheet not in ['ensemble', 'glossaire']:
        df = pd.read_excel(files[0], sheet, index_col=None)
        if sheet != 'donneur':
            sheet = '{}_{}'.format('patient', sheet)        
        info = {
        sheet : list(df.columns.values)
        }
        output.update(info)
DIM_MAP_FIELDS = output
output.keys()

dict_keys(['patient_preoperatoire', 'donneur', 'patient_intraoperatoire', 'patient_postoperatoire'])

In [12]:
for name, array in DIM_MAP_FIELDS.items():
    df = df[df.columns.intersection(array)]
    filename = os.path.join('', 'dim_{}.csv'.format(name))
    df.to_csv(filename)